In [ ]:

import pandas as pd
import numpy as np
from numpy import random
random.seed(1998)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ebay
#%cd /content/drive/MyDrive/tempShare
df = pd.read_csv('eBay_ML_Challenge_Dataset_2021_train.tsv', sep='\t')
quiz = pd.read_csv('eBay_ML_Challenge_Dataset_2021_quiz.tsv', sep='\t')

Mounted at /content/drive
/content/drive/MyDrive/ebay


In [ ]:
pip install pgeocode

In [ ]:
import pgeocode
import xgboost as xgb
from sklearn.metrics import mean_squared_error


In [ ]:
df

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.00,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.00,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.50,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.00,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.00,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,B2C,80139,5.0,2018-05-04 15:36:00.000-05:00,1,9.85,2,5,60010,08846,3,7.35,1,2018-04-30 22:30:00.000-05:00,2018-05-07,32,1,PACKAGE_THICK_ENVELOPE,14999996
14999996,C2C,452486,2.0,2018-05-02 14:53:00.000-04:00,2,0.00,2,9,47909,29070,1,36.00,1,2018-05-01 19:59:43.000-04:00,2018-05-08,64,1,PACKAGE_THICK_ENVELOPE,14999997
14999997,B2C,43307,3.0,2018-06-06 15:21:00.000-07:00,2,2.99,2,9,91766,62363,0,7.99,1,2018-06-05 14:20:14.000-07:00,2018-06-11,0,1,NONE,14999998
14999998,B2C,24235,1.0,2018-12-17 10:55:00.000-05:00,0,0.00,3,5,18011,000,1,34.99,1,2018-12-16 15:01:46.000-05:00,2018-12-20,13,1,PACKAGE_THICK_ENVELOPE,14999999


#Preprocessing Data and feature engineering

In [ ]:
dist = pgeocode.GeoDistance('US')
postal = pgeocode.Nominatim('US')

def calculate_distance(subset):
  x = subset['buyer_zip_new']
  y = subset['item_zip_new']
  distance = dist.query_postal_code(x.to_numpy(), y.to_numpy())
  subset['distance'] = pd.DataFrame({'col': distance})
  return subset

def return_state(subset):
  x = subset['buyer_zip_new']
  y = subset['item_zip_new']
  buyer_states = postal.query_postal_code(x.to_numpy())['state_code']
  item_states = postal.query_postal_code(y.to_numpy())['state_code']
  #the adding latitude is here, in case you need to delete it in the future
  item_lat = postal.query_postal_code(y.to_numpy())['latitude']
  item_long = postal.query_postal_code(y.to_numpy())['longitude']
  buyer_lat = postal.query_postal_code(x.to_numpy())['latitude']
  buyer_long = postal.query_postal_code(x.to_numpy())['longitude']

  return buyer_states.tolist(), item_states.tolist(), item_lat.tolist(), item_long.tolist(), buyer_lat.tolist(), buyer_long.tolist() 


  
def calculating_duration(df):
  df['delivery_date2'] = df['delivery_date'].apply(lambda x: x[:10])
  #convert to datetime for further calculation
  df['delivery_date2'] = pd.to_datetime(df['delivery_date2'])
  #first have to check if there is incorrect data in the delivery time and payment time
  df['duration'] = df['delivery_date2'] - df['acceptance_scan_time2']
  df['duration'] = df.loc[:, 'duration'].dt.days
  return df

def assign_state_code(df):
  buyer, item,item_lat, item_long, buyer_lat, buyer_long = return_state(df)
  df['buyer_state'] = pd.DataFrame({'col': buyer})
  df['item_state'] = pd.DataFrame({'col': item})
  '''here you add the lattitude and longtitude code'''
  df['item_lat'] = pd.DataFrame({'col': item_lat})
  df['item_long'] = pd.DataFrame({'col': item_long})
  df['buyer_lat'] = pd.DataFrame({'col': buyer_lat})
  df['buyer_long'] = pd.DataFrame({'col': buyer_long})
  return df

# 25% quantile 
def q25(x):
    return x.quantile(0.25)
# 75th Percentile
def q75(x):
    return x.quantile(0.75)



In [ ]:
def preprocessing_data(df):
  #calculate the time zone difference between seller and buyer
  df['time_zone_difference'] = df['acceptance_scan_timestamp'].apply(lambda x: x[-6:-3]).astype(int) - df['payment_datetime'].apply(lambda x: x[-6:-3]).astype(int) 
  #calculate the seller hours when the payment is being made
  df['seller_hour'] = df['payment_datetime'].apply(lambda x: x[11:13]).astype(int) + df['time_zone_difference'] 
  #check if the seller hour is after 2 pm
  lst_2pm = df['seller_hour'] >= 14
  df['larger_than_2pm'] = pd.DataFrame({'col': lst_2pm})
  #data cleaning for the time data
  df['acceptance_scan_time2'] = df['acceptance_scan_timestamp'].apply(lambda x: x[:10])
  df['payment_date2'] = df['payment_datetime'].apply(lambda x: x[:10])
  #convert to datetime for further calculation
  df['payment_date2'] = pd.to_datetime(df['payment_date2'])
  df['acceptance_scan_time2'] = pd.to_datetime(df['acceptance_scan_time2'])
  # extract day of month, year, 
  df['pay_month'] = df.payment_date2.dt.month
  df['pay_year'] = df.payment_date2.dt.year
  df['scan_month'] = df.acceptance_scan_time2.dt.month

  #extract daytime of the payment time, and acceptance time
  df['payment_hour'] = df['payment_datetime'].apply(lambda x: x[11:13]).astype(int)
  df['scan_hour'] = df['acceptance_scan_timestamp'].apply(lambda x: x[11:13]).astype(int)

  #extract the day of the week of payment time and acceptance time
  df['Weekday_scan'] = df['acceptance_scan_time2'].dt.weekday
  df['Weekday_pay'] = df['payment_date2'].dt.weekday


  #extract quarter of the payment
  df['payment_quarter'] = df['payment_date2'].dt.quarter
  df['min_delivery'] = df['carrier_min_estimate'] + df['declared_handling_days'] #new feature
  df['max_delivery'] = df['carrier_max_estimate'] + df['declared_handling_days'] #new feature

  #convert the zip code to string 
  df['buyer_zip_new'] = df['buyer_zip'].astype(str).apply(lambda x: x[:5])
  df['item_zip_new'] = df['item_zip'].astype(str).apply(lambda x: x[:5])
  
  #convert the categorical data to boolean type of data
  df['b2c_c2c'] = df['b2c_c2c'].astype(str)
  lst_b2b = df.iloc[:, 0] == 'B2C'
  df['B2B'] = pd.DataFrame({'col': lst_b2b})

  #return the state_code of each item and customer

  '''here you also add the lattitude and longtitude'''

  assign_state_code(df)


  #calculate the distance between item and buyer
  df = calculate_distance(df)

  #assign state code to each item and customer
  df = assign_state_code(df)
  
  #check if there is miss-matching date for delivery and payment
  package_dic = {'NONE': 0, 'LETTER': 1, 'PACKAGE_THICK_ENVELOPE': 2, 'LARGE_ENVELOPE': 3, 'LARGE_PACKAGE': 4, 'EXTRA_LARGE_PACKAGE': 5,  'VERY_LARGE_PACKAGE': 6}
  df.loc[:, 'size_ordinal'] = df.package_size.map(package_dic)

  return df

In [ ]:
def adding_features(df):
  #check if the customer is making a purchase during weekends, if the carrier is scanning/working on weekends
  #since the Friday = 4, Saturday = 5, and Sunday = 6
  df['is_payment_weekend'] = df['Weekday_pay'] > 4
  df['is_scan_weeknd'] = df['Weekday_scan'] > 4

  return df

In [ ]:
df = preprocessing_data(df)
quiz = preprocessing_data(quiz)
df = adding_features(df)
quiz = adding_features(quiz)

#adding our target variable to the dataframe, which is the duration between the carrier scanning time and the delivery date
df = calculating_duration(df)


In [ ]:
#we can find that the there is a lot of missing distance(179057 data)
#this is partially because there are international customers and sellers
#we are gonna use a different method to get their distance

missing_dist = df[df['distance'].isnull()]



In [ ]:
US_state = df['buyer_state'].unique()
US_state

array(['MI', 'IL', 'WI', 'CO', 'MN', 'TX', 'SC', 'FL', 'UT', 'MD', 'NC',
       'OH', 'MO', 'CA', 'PA', 'ID', 'NY', 'KS', 'GA', 'ME', 'KY', 'VA',
       'MA', 'AZ', 'AL', 'NJ', 'MT', 'CT', 'WY', 'OK', 'LA', 'IN', 'TN',
       'NE', 'WA', 'NM', 'MS', 'AR', 'OR', 'IA', 'WV', 'VT', 'NV', nan,
       'ND', 'SD', 'DC', 'NH', 'DE', 'RI', 'AK', 'HI'], dtype=object)

In [ ]:
#sometimes the US states will have the same abbreviation as other countries' country code
for i in range(52):
  if US_state[i] in missing_state:
    print("collision detect")
    print(US_state[i])

NameError: ignored

In [ ]:
df['Is_within_US'] = df['distance'].notnull()


In [ ]:
quiz['Is_within_US'] = quiz['distance'].notnull()

In [ ]:
'''

def get_country_code(zip_code):
  
  for i in pgeocode.COUNTRIES_VALID:
    nomi = pgeocode.Nominatim(i)
   # this code will always return a dataframe
   # if the postal_code has no match, the df values will all be NaN 
    df = nomi.query_postal_code(zip_code)
    if type(df.country_code) == str:
      return df.country_code, df.latitude, df.longitude

def get_location_for_array(lst):
  state_name = []
  latitude = []
  longtitude = []
  for i in lst:
    try: 
      state, la, long = get_country_code(i)
      state_name.append(state)
      latitude.append(la)
      longtitude.append(long)
      print('{} th one is done'.format(i))
    except:
      state_name.append(float('nan'))
      latitude.append(float('nan'))
      longtitude.append(float('nan'))
      print('{} th one is done'.format(i))
    
  info = { 'missing_zip': lst , 'state_name' : state_name,
        'latitude': latitude,
        'longtitude': longtitude
       }

  df_new = pd.DataFrame.from_dict(info)
  df_new.to_csv('missing_new.csv')
  return df_new

'''

In [ ]:
df = df.loc[df['duration'] >= 0 ]

In [ ]:
df

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number,time_zone_difference,seller_hour,larger_than_2pm,acceptance_scan_time2,payment_date2,pay_month,pay_year,scan_month,payment_hour,scan_hour,Weekday_scan,Weekday_pay,payment_quarter,min_delivery,max_delivery,buyer_zip_new,item_zip_new,B2B,buyer_state,item_state,distance,size_ordinal,delivery_date2,duration
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.00,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1,0,3,False,2019-03-26,2019-03-24,3,2019,3,3,15,1,6,1,6.0,8.0,49040,97219,True,MI,OR,3001.984391,1,2019-03-29,3
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.00,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2,0,13,False,2018-06-02,2018-06-01,6,2018,6,13,12,5,4,2,5.0,7.0,62521,11415,False,IL,NY,1284.515542,2,2018-06-05,3
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.50,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3,0,0,False,2019-01-07,2019-01-06,1,2019,1,0,16,0,6,1,4.0,6.0,53010,27292,True,WI,NC,1102.546134,2,2019-01-10,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.00,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4,0,10,False,2018-12-17,2018-12-16,12,2018,12,10,16,0,6,4,4.0,6.0,80022,90703,True,CO,CA,1343.238967,2,2018-12-21,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.00,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5,0,18,True,2018-07-27,2018-07-26,7,2018,7,18,16,4,3,3,4.0,6.0,55070,90703,True,MN,CA,2458.659875,2,2018-07-30,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,B2C,80139,5.0,2018-05-04 15:36:00.000-05:00,1,9.85,2,5,60010,08846,3,7.35,1,2018-04-30 22:30:00.000-05:00,2018-05-07,32,1,PACKAGE_THICK_ENVELOPE,14999996,0,22,True,2018-05-04,2018-04-30,4,2018,5,22,15,4,0,2,7.0,10.0,08846,60010,True,NJ,IL,1150.326675,2,2018-05-07,3
14999996,C2C,452486,2.0,2018-05-02 14:53:00.000-04:00,2,0.00,2,9,47909,29070,1,36.00,1,2018-05-01 19:59:43.000-04:00,2018-05-08,64,1,PACKAGE_THICK_ENVELOPE,14999997,0,19,True,2018-05-02,2018-05-01,5,2018,5,19,14,2,1,2,4.0,11.0,29070,47909,False,SC,IN,859.593302,2,2018-05-08,6
14999997,B2C,43307,3.0,2018-06-06 15:21:00.000-07:00,2,2.99,2,9,91766,62363,0,7.99,1,2018-06-05 14:20:14.000-07:00,2018-06-11,0,1,NONE,14999998,0,14,True,2018-06-06,2018-06-05,6,2018,6,14,15,2,1,2,5.0,12.0,62363,91766,True,IL,CA,2466.969970,0,2018-06-11,5
14999998,B2C,24235,1.0,2018-12-17 10:55:00.000-05:00,0,0.00,3,5,18011,000,1,34.99,1,2018-12-16 15:01:46.000-05:00,2018-12-20,13,1,PACKAGE_THICK_ENVELOPE,14999999,0,15,True,2018-12-17,2018-12-16,12,2018,12,15,10,0,6,4,4.0,6.0,000,18011,True,NaN,PA,NaN,2,2018-12-20,3


In [ ]:
df_subset = df.loc[:, ['buyer_state', 'item_state', 'duration']]
df1 = df_subset.groupby(['buyer_state', 'item_state']).duration.agg(['max', 'min','mean', q25, 'median', q75])


In [ ]:
df = df.join(df1, on=['buyer_state', 'item_state'], how = 'left')
quiz = quiz.join(df1, on = ['buyer_state', 'item_state'], how = 'left')

#Train the XGboost

In [ ]:
import pandas as pd
import numpy as np
from numpy import random
random.seed(1998)
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ebay

Mounted at /content/drive
/content/drive/MyDrive/ebay


In [ ]:
##If needed to load in data, here is the way
df = pd.read_hdf('Xgboost_trainingdata.h5')
quiz = pd.read_hdf('Xgboost_quiz.h5')

In [ ]:
df.columns

Index(['Is_within_US', 'is_payment_weekend', 'is_scan_weeknd', 'item_lat',
       'item_long', 'buyer_lat', 'buyer_long', 'B2B', 'seller_id',
       'declared_handling_days', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'category_id',
       'item_price', 'quantity', 'weight', 'weight_units',
       'time_zone_difference', 'seller_hour', 'larger_than_2pm', 'pay_month',
       'pay_year', 'scan_month', 'payment_hour', 'scan_hour', 'Weekday_scan',
       'Weekday_pay', 'payment_quarter', 'min_delivery', 'max_delivery',
       'distance', 'max', 'min', 'mean', 'q25', 'median', 'q75',
       'size_ordinal', 'duration'],
      dtype='object')

In [ ]:

X_quiz = quiz[['Is_within_US','is_payment_weekend', 'is_scan_weeknd','item_lat', 'item_long', 'buyer_lat', 'buyer_long','B2B','seller_id' ,'declared_handling_days','shipment_method_id','shipping_fee','carrier_min_estimate','carrier_max_estimate','category_id','item_price','quantity','weight','weight_units','time_zone_difference','seller_hour','larger_than_2pm','pay_month','pay_year','scan_month','payment_hour','scan_hour','Weekday_scan','Weekday_pay','payment_quarter','min_delivery','max_delivery','distance','max','min'	,'mean'	,'q25',	'median','q75','size_ordinal']]

In [ ]:
X = df[['Is_within_US','is_payment_weekend', 'is_scan_weeknd','item_lat', 'item_long', 'buyer_lat', 'buyer_long','B2B','seller_id' ,'declared_handling_days','shipment_method_id','shipping_fee','carrier_min_estimate','carrier_max_estimate','category_id','item_price','quantity','weight','weight_units','time_zone_difference','seller_hour','larger_than_2pm','pay_month','pay_year','scan_month','payment_hour','scan_hour','Weekday_scan','Weekday_pay','payment_quarter','min_delivery','max_delivery','distance','max','min'	,'mean'	,'q25',	'median','q75','size_ordinal']]
y = df[['duration']]



In [ ]:
#X_quiz.to_hdf('Xgboost_quiz.h5', key = 'X_quiz', mode = 'w')

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.5, learning_rate = 0.1,
                max_depth = 7, alpha = 10, n_estimators = 100)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[22:24:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:24:18] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
RMSE: 1.419268


In [ ]:
print("RMSE: %f" % (rmse))

RMSE: 1.425460


In [ ]:
def loss(preds, y_test):
  new_y = y_test.values.flatten()
  result = new_y - preds
  late = 0
  early = 0
  for i in range(len(result)):
    if result[i] > 0:
      late = late + result[i]
    elif result[i] < 0:
      early = early + abs(result[i])
  
  total = (early*0.4 + late*0.6)/len(result)
  return total

In [ ]:
print("LOSS: %f" % (loss(preds, y_test)))


LOSS: 0.420468


In [ ]:
def rounding(arr, value):
  new = np.zeros(np.shape(arr))
  for i in range(len(arr)):
    if arr[i] - arr[i]//1 <= value:
      new[i] = arr[i]//1
    else:
      new[i] = arr[i]//1 + 1
  return new

In [ ]:
#see what will be the optimal rounding rule
grid = [0.62, 0.63, 0.65,0.66]
def find_optimal_rounding(preds, y_test):
  for i in range(len(grid)):
    new_pred = rounding(preds, grid[i])
    print("LOSS: %f" % (loss(new_pred, y_test)))


In [ ]:
#this seems to be not working

def loss(preds, dtrain):
    ''' Customized loss function designing to punish earlier delivery date more than later delivery date'''
    y = dtrain.get_label()
    early = (sum(abs(preds[preds < y] - y[preds < y])))*0.4
    late = (sum(abs(preds[preds > y] - y[preds < y])))*0.6
    loss = (early + late)/len(y)
    return 'loss', loss

In [ ]:
'''params = {"objective":"reg:linear",'colsample_bytree': 0.5,'learning_rate': 0.1,
                'max_depth':7, 'alpha': 10, 'n_estimators': 100, 'num_boost_round':100 } # 'disable_default_eval_metric': 1

xg_reg = xgb.train(params=params, dtrain=data_dmatrix)'''
'''cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                , early_stopping_rounds=10,metrics="rmse"
          as_pandas=True, seed=123)'''

[01:24:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:24:01] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


'cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,\n                , early_stopping_rounds=10,metrics="rmse"\n          as_pandas=True, seed=123)'

In [ ]:
preds_quiz = xg_reg.predict(xgb.DMatrix(X_quiz))

In [ ]:
#use this if did not use train
preds_quiz = xg_reg.predict(X_quiz)

AttributeError: ignored

In [ ]:
preds_round = preds_quiz.round()
quiz['duration'] = pd.DataFrame({'col': preds_round.tolist()})
quiz['delivery_date'] = quiz['acceptance_scan_time2'] + pd.to_timedelta(quiz['duration'], unit='D')
quiz['delivery_date'] = quiz['delivery_date'].astype(str)
submit = quiz[['record_number','delivery_date']]
submit.to_csv("submit_1118_depth7_max10_n70_.tsv.gz", 
           index=False, sep='\t', header = False, compression = 'gzip')

In [ ]:
def rounding_rule(arr):
  new = np.zeros(np.shape(arr))
  for i in range(len(arr)):
    if arr[i] - arr[i]//1 <= 0.91:
      new[i] = arr[i]//1
    else:
      new[i] = arr[i]//1 + 1
  return new



In [ ]:
quiz = pd.read_csv('eBay_ML_Challenge_Dataset_2021_quiz.tsv', sep='\t')

In [ ]:

quiz['acceptance_scan_time2'] = quiz['acceptance_scan_timestamp'].apply(lambda x: x[:10])
quiz['acceptance_scan_time2'] = pd.to_datetime(quiz['acceptance_scan_time2'])

In [ ]:
preds_new = rounding_rule(preds_quiz)
quiz['duration_91'] = pd.DataFrame({'col': preds_new.tolist()})
quiz['delivery_date_91'] = quiz['acceptance_scan_time2'] + pd.to_timedelta(quiz['duration_91'], unit='D')
submit_new = quiz[['record_number','delivery_date_91']]
submit_new.to_csv("submit_0106_depth7_max10_n100_cosample05_round91_trainwholedataset.tsv.gz", 
           index=False, sep='\t', header = False, compression = 'gzip')

In [ ]:
#the minimum loss
#is when you only merge on data ['buyer_state', 'item_state']
#max_depth = 7, alpha = 10, number_estimators = 100, rounding = 83